# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [163]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv
# Import API key
from api_keys import g_key

import json

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [164]:
# Load the csv 
weather_city_file = "../WeatherPy/Output/weather_city.csv"

# exported the csv file loaded to a DataFrame
weather_cities_df = pd.read_csv(weather_city_file)
weather_cities_df

,city,latitude,longitude,temp_max,humidity,cloudiness,wind_speed,country,date
0,Grand Gaube,-20.01,57.66,63.00,64,0,1.99,MU,1604107667
1,Saint George,37.10,-113.58,69.80,15,1,3.36,US,1604107472
2,Muisne,0.60,-80.03,75.18,83,100,7.11,EC,1604107667
3,Ushuaia,-54.80,-68.30,50.00,71,75,18.34,AR,1604107667
4,Mandalgovi,45.76,106.27,27.27,77,100,14.14,MN,1604107668
...,...,...,...,...,...,...,...,...,...
477,Dali,25.70,100.18,59.07,85,65,2.39,CN,1604107744
478,Hudson Bay,52.85,-102.38,33.93,91,100,10.09,CA,1604107744
479,Khilok,51.37,110.47,25.16,95,100,9.17,RU,1604107745
480,Oranjestad,12.52,-70.03,84.20,79,40,8.05,AW,1604107694


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [165]:
#Configure gmaps
gmaps.configure(api_key=g_key)

In [166]:
# handle NaN values
weather_cities_df = weather_cities_df.dropna()

# Store 'Lat' and 'Lng' into  locations 
locations = weather_cities_df[['latitude','longitude']].astype(float)

# Convert Humidity to float and store
humidity = weather_cities_df['humidity'].astype(float)
weather_cities_df

,city,latitude,longitude,temp_max,humidity,cloudiness,wind_speed,country,date
0,Grand Gaube,-20.01,57.66,63.00,64,0,1.99,MU,1604107667
1,Saint George,37.10,-113.58,69.80,15,1,3.36,US,1604107472
2,Muisne,0.60,-80.03,75.18,83,100,7.11,EC,1604107667
3,Ushuaia,-54.80,-68.30,50.00,71,75,18.34,AR,1604107667
4,Mandalgovi,45.76,106.27,27.27,77,100,14.14,MN,1604107668
...,...,...,...,...,...,...,...,...,...
477,Dali,25.70,100.18,59.07,85,65,2.39,CN,1604107744
478,Hudson Bay,52.85,-102.38,33.93,91,100,10.09,CA,1604107744
479,Khilok,51.37,110.47,25.16,95,100,9.17,RU,1604107745
480,Oranjestad,12.52,-70.03,84.20,79,40,8.05,AW,1604107694


In [167]:

# fig gmaps
fig = gmaps.figure()

# Create a humidity Heatmap layer
humidity_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                     dissipating=False, max_intensity=5,
                                     point_radius = 1)
#fig
fig.add_layer(humidity_layer)



In [93]:
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [168]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.

# narrows down the cities to fit weather conditions.
weather_ideal_df = weather_cities_df.loc[(weather_cities_df['temp_max']>70)&(weather_cities_df['temp_max']<80)&
                                        (weather_cities_df['cloudiness']==0)&(weather_cities_df['humidity']<60),:]

# handle NaN values
weather_ideal_df = weather_ideal_df.dropna()

weather_ideal_df


,city,latitude,longitude,temp_max,humidity,cloudiness,wind_speed,country,date
102,Esperance,-33.87,121.90,75.20,31,0,25.28,AU,1604107684
205,Arlit,18.74,7.39,75.49,19,0,11.59,NE,1604107701
214,Airai,-8.93,125.41,75.87,53,0,2.57,TL,1604107512
215,Ximeicun,24.96,118.38,71.60,53,0,15.66,CN,1604107702
239,Umluj,25.02,37.27,77.25,37,0,4.99,SA,1604107706
310,Ormara,25.21,64.64,75.76,33,0,6.55,PK,1604107718
345,Kaura Namoda,12.59,6.58,78.49,24,0,7.76,NG,1604107724
352,Salalah,17.02,54.09,73.40,40,0,5.82,OM,1604107725
430,Morondava,-20.28,44.28,75.94,48,0,13.98,MG,1604107737
450,San Lorenzo,-25.33,-57.53,71.60,43,0,16.11,PY,1604107740


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [169]:
#stores into variable named hotel_df
hotel_df = pd.DataFrame(weather_ideal_df, columns=['city','country','latitude','longitude','Hotel Name'])
hotel_df

,city,country,latitude,longitude,Hotel Name
102,Esperance,AU,-33.87,121.90,NaN
205,Arlit,NE,18.74,7.39,NaN
214,Airai,TL,-8.93,125.41,NaN
215,Ximeicun,CN,24.96,118.38,NaN
239,Umluj,SA,25.02,37.27,NaN
310,Ormara,PK,25.21,64.64,NaN
345,Kaura Namoda,NG,12.59,6.58,NaN
352,Salalah,OM,17.02,54.09,NaN
430,Morondava,MG,-20.28,44.28,NaN
450,San Lorenzo,PY,-25.33,-57.53,NaN


In [ ]:
# add columns for the hotel data retrieval
hotel_df['Hotel Name'] = ''
hotel_df['Lat'] = ''
hotel_df['Lng'] = ''

# Setting parameters to search for hotels with 5000 meters               
target_search = "hotel"
target_radius = 5000
target_type = "hotel"

In [125]:
# Loop through the hotel_df and run a lat/long search for each city
for index, row in hotel_df.iterrows():
    
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
     
    #get the latitude and longitude for the city
    latitude  = row['latitude']
    longitude = row['longitude']

    #save lat and long for params
    target_coordinates = f"{latitude},{longitude}"

    #setting up a parameters to search for hotels with 5000 meters
    params = {
                "location": target_coordinates,
                "keyword": target_search,
                "radius": target_radius,
                "type": target_type,
                "key": g_key
            }
 
    try:
        # run a request using the params dictionary
        response = requests.get(base_url, params=params)

        # convert response to json
        hotels_data = response.json()
        
        #getting hotel data from google search
        v_hotel =hotels_data['results'][0]['name']
        v_hotel_lat =hotels_data['results'][0]["geometry"]['location']['lat']
        v_hotel_long =hotels_data['results'][0]["geometry"]['location']['lng']
        print(v_hotel)
        
        #assigning the obtained value to each row of dataframe
        hotel_df.at[index, 'Hotel Name'] =  v_hotel
        hotel_df.at[index, 'Lat'] =  v_hotel_lat
        hotel_df.at[index, 'Lng'] =  v_hotel_long

    except:
        pass 


Hôtel Telwa Bungalow
18.7389762
7.389448199999999
Fanhua Hotel
24.962133
118.415823
Shada Collection Mina Hotel
25.04203
37.25263
Alqadri hotel zero point ormara
25.2492975
64.5966851
Moonlight Hotel
12.5938473
6.589940700000001
Salalah Gardens Hotel
17.022848
54.064693
Palissandre Cote Ouest resort & SPA
-20.3000259
44.2679367
Dazzler by Wyndham Asunción
-25.284773
-57.568288
Sails in the Desert
-25.2390223
130.9834051
THE SOMNATH GATEWAY
20.901515
70.4059475


,city,country,latitude,longitude,Hotel Name,Lat,Lng
102,Esperance,AU,-33.87,121.90,,,
205,Arlit,NE,18.74,7.39,Hôtel Telwa Bungalow,18.739,7.38945
214,Airai,TL,-8.93,125.41,,,
215,Ximeicun,CN,24.96,118.38,Fanhua Hotel,24.9621,118.416
239,Umluj,SA,25.02,37.27,Shada Collection Mina Hotel,25.042,37.2526
310,Ormara,PK,25.21,64.64,Alqadri hotel zero point ormara,25.2493,64.5967
345,Kaura Namoda,NG,12.59,6.58,Moonlight Hotel,12.5938,6.58994
352,Salalah,OM,17.02,54.09,Salalah Gardens Hotel,17.0228,54.0647
430,Morondava,MG,-20.28,44.28,Palissandre Cote Ouest resort & SPA,-20.3,44.2679
450,San Lorenzo,PY,-25.33,-57.53,Dazzler by Wyndham Asunción,-25.2848,-57.5683


In [129]:
# rename the columns city and country for the marker_layer
hotel_df.rename(columns={'city':'City','country':'Country'},inplace=True)
hotel_df

,City,Country,latitude,longitude,Hotel Name,Lat,Lng
102,Esperance,AU,-33.87,121.90,,,
205,Arlit,NE,18.74,7.39,Hôtel Telwa Bungalow,18.739,7.38945
214,Airai,TL,-8.93,125.41,,,
215,Ximeicun,CN,24.96,118.38,Fanhua Hotel,24.9621,118.416
239,Umluj,SA,25.02,37.27,Shada Collection Mina Hotel,25.042,37.2526
310,Ormara,PK,25.21,64.64,Alqadri hotel zero point ormara,25.2493,64.5967
345,Kaura Namoda,NG,12.59,6.58,Moonlight Hotel,12.5938,6.58994
352,Salalah,OM,17.02,54.09,Salalah Gardens Hotel,17.0228,54.0647
430,Morondava,MG,-20.28,44.28,Palissandre Cote Ouest resort & SPA,-20.3,44.2679
450,San Lorenzo,PY,-25.33,-57.53,Dazzler by Wyndham Asunción,-25.2848,-57.5683


In [137]:
#we discard the cities that do not have hotels around 5000 meters
hotel_df = hotel_df.loc[(hotel_df['Hotel Name'] != '')]
hotel_df

,City,Country,latitude,longitude,Hotel Name,Lat,Lng
205,Arlit,NE,18.74,7.39,Hôtel Telwa Bungalow,18.739,7.38945
215,Ximeicun,CN,24.96,118.38,Fanhua Hotel,24.9621,118.416
239,Umluj,SA,25.02,37.27,Shada Collection Mina Hotel,25.042,37.2526
310,Ormara,PK,25.21,64.64,Alqadri hotel zero point ormara,25.2493,64.5967
345,Kaura Namoda,NG,12.59,6.58,Moonlight Hotel,12.5938,6.58994
352,Salalah,OM,17.02,54.09,Salalah Gardens Hotel,17.0228,54.0647
430,Morondava,MG,-20.28,44.28,Palissandre Cote Ouest resort & SPA,-20.3,44.2679
450,San Lorenzo,PY,-25.33,-57.53,Dazzler by Wyndham Asunción,-25.2848,-57.5683
464,Yulara,AU,-25.24,130.99,Sails in the Desert,-25.239,130.983
471,Verāval,IN,20.90,70.37,THE SOMNATH GATEWAY,20.9015,70.4059


In [156]:
 # NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index,row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [159]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig



Figure(layout=FigureLayout(height='420px'))